In [201]:
# Import packages we will need and set working directory:
import pandas as pd
import json
import csv
import datetime


file_path = r"C:/Users/mfazekas/Documents/Python Work/"

In [48]:
# Let's take a quick look at the Users table to see what we're working with, reading in the file with no changes but filling blanks, the dump to csv
users_filename = "users.json"
user_table = pd.read_json(file_path + users_filename, lines = True)
user_table.fillna(0,inplace=True)
user_table.to_csv(file_path + 'user_table.csv', index = False)

# Let's repeat for Receipts - as Excel since the recipts items are not behaving due to plaintext descriptions:
receipts_filename = "receipts.json"
receipts_table = pd.read_json(file_path + receipts_filename, lines = True)
receipts_table.fillna(0,inplace=True)
receipts_table.to_excel(file_path + 'receipts_table.xlsx', sheet_name = 'data', index = False)

# And Brands:
brands_filename = "brands.json"
brands_table = pd.read_json(file_path + brands_filename, lines = True)
brands_table.fillna(0,inplace=True)
brands_table.to_csv(file_path + 'brands_table.csv', index = False)

In [22]:
# Observations:

# The Users table is fairly clean, only having some IDs that need to be parsed out, and dates that need to be parsed and converted to human-readable times. 
# As far as data quality issues - some Signup Sources are null as well as States. A few records that are identical, mostly Fetch Staff accounts.

# The Brands table is not as good. Lots of null values for category/CategoryCode/BrandCode, and a large number of test values if this is a production system. 
# CPG field needs to be broken out into ID and Ref. Some refs are 'Cogs', some are 'Cpgs' - unclear what this means. 

# Receipts table needs the most work. In addition to dates/ID fields that need to be parsed out, the rewardsReceiptItemList is deserving of its own table due to the nature of 
# and amount of information present.  

In [97]:
# After some more review - the JSON files are actually malformed and produced confusing results. Using the code below fixes these issues by converting the files to proper lists of objects:

from pprint import pprint

with open(r"C:/Users/mfazekas/Documents/Python Work/users.json") as f:
    users = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")

In [129]:
# And then we use this package to make the output pretty & take care of nested references:
users2 = pd.json_normalize(users)

In [130]:
# Do some renaming/date conversions on my new users table:

users2.rename(columns = {'_id.$oid': 'ID', 'createdDate.$date':'createdDate', 'lastLogin.$date':'lastLogin'}, inplace = True)
users2['createdDate'] = pd.to_datetime(users2.createdDate, unit='ms')
users2['lastLogin'] = pd.to_datetime(users2.lastLogin, unit='ms')
users2

,active,role,signUpSource,state,ID,createdDate,lastLogin
0,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
1,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
2,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
3,True,consumer,Email,WI,5ff1e1eacfcf6c399c274ae6,2021-01-03 15:25:30.554,2021-01-03 15:25:30.597
4,True,consumer,Email,WI,5ff1e194b6a9d73a3a9f1052,2021-01-03 15:24:04.800,2021-01-03 15:25:37.858
...,...,...,...,...,...,...,...
490,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204
491,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204
492,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204
493,True,fetch-staff,NaN,NaN,54943462e4b07e684157a532,2014-12-19 14:21:22.381,2021-03-05 16:52:23.204


In [134]:
# Now let's tackle the brands table: 
from pprint import pprint

with open(r"C:/Users/mfazekas/Documents/Python Work/brands.json") as f:
    brands = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")
    
brands2 = pd.json_normalize(brands)

,barcode,category,categoryCode,name,topBrand,_id.$oid,cpg.$id.$oid,cpg.$ref,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs,NaN
1163,511111706328,Breakfast & Cereal,NaN,Dippin DotsÂ® Cereal,NaN,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs,DIPPIN DOTS CEREAL
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1598639215217
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,False,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs,LIPTON TEA Leaves


In [138]:
# No dates to format here, just some column name cleanup:
brands2.rename(columns = {'_id.$oid': 'ID', 'cpg.$id.$oid':'cpgID', 'cpg.$ref':'cpgRef'}, inplace = True)
brands2

,barcode,category,categoryCode,name,topBrand,ID,cpgID,cpgRef,brandCode
0,511111019862,Baking,BAKING,test brand @1612366101024,False,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs,NaN
1,511111519928,Beverages,BEVERAGES,Starbucks,False,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs,STARBUCKS
2,511111819905,Baking,BAKING,test brand @1612366146176,False,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146176
3,511111519874,Baking,BAKING,test brand @1612366146051,False,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs,TEST BRANDCODE @1612366146051
4,511111319917,Candy & Sweets,CANDY_AND_SWEETS,test brand @1612366146827,False,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1612366146827
...,...,...,...,...,...,...,...,...,...
1162,511111116752,Baking,BAKING,test brand @1601644365844,NaN,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs,NaN
1163,511111706328,Breakfast & Cereal,NaN,Dippin DotsÂ® Cereal,NaN,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs,DIPPIN DOTS CEREAL
1164,511111416173,Candy & Sweets,CANDY_AND_SWEETS,test brand @1598639215217,NaN,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs,TEST BRANDCODE @1598639215217
1165,511111400608,Grocery,NaN,LIPTON TEA Leaves,False,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs,LIPTON TEA Leaves


In [195]:
# Now time for receipts: 
from pprint import pprint

with open(r"C:/Users/mfazekas/Documents/Python Work/receipts.json") as f:
    receipts = json.loads("[" + 
        f.read().replace("}\n{", "},\n{") + 
    "]")
    
receipts2 = pd.json_normalize(receipts)

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,_id.$oid,createDate.$date,dateScanned.$date,finishedDate.$date,modifyDate.$date,pointsAwardedDate.$date,purchaseDate.$date
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,1.609632e+12
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,1.609601e+12
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,1609687537000,1609687537000,NaN,1609687542000,NaN,1.609632e+12
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,1.609632e+12
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,1.609601e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603cc0630a720fde100003e6,1614594147000,1614594147000,NaN,1614594148000,NaN,1.597622e+12
1115,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603d0b710a720fde1000042a,1614613361873,1614613361873,NaN,1614613361873,NaN,NaN
1116,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603cf5290a720fde10000413,1614607657664,1614607657664,NaN,1614607657664,NaN,NaN
1117,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603ce7100a7217c72c000405,1614604048000,1614604048000,NaN,1614604049000,NaN,1.597622e+12


In [196]:
# This table has some dates to clean up... as well as the nested rewardsReceiptItemList object that needs to be broken out. 

# rename and convert times:
receipts2.rename(columns = {'_id.$oid': 'ID', 
                            'createDate.$date':'createDate', 
                            'dateScanned.$date':'dateScanned', 
                            'finishedDate.$date': 'finishedDate', 
                            'modifyDate.$date': 'modifyDate', 
                            'pointsAwardedDate.$date': 'pointsAwardedDate', 
                            'purchaseDate.$date': 'purchaseDate'}, inplace = True)
receipts2['createDate'] = pd.to_datetime(receipts2.createDate, unit='ms')
receipts2['dateScanned'] = pd.to_datetime(receipts2.dateScanned, unit='ms')
receipts2['finishedDate'] = pd.to_datetime(receipts2.finishedDate, unit='ms')
receipts2['modifyDate'] = pd.to_datetime(receipts2.modifyDate, unit='ms')
receipts2['pointsAwardedDate'] = pd.to_datetime(receipts2.pointsAwardedDate, unit='ms')
receipts2['purchaseDate'] = pd.to_datetime(receipts2.purchaseDate, unit='ms')
receipts2

,bonusPointsEarned,bonusPointsEarnedReason,pointsEarned,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId,ID,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,purchaseDate
0,500.0,"Receipt number 2 completed, bonus point schedu...",500.0,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6,5ff1e1eb0a720f0523000575,2021-01-03 15:25:31.000,2021-01-03 15:25:31.000,2021-01-03 15:25:31,2021-01-03 15:25:36.000,2021-01-03 15:25:31,2021-01-03 00:00:00
1,150.0,"Receipt number 5 completed, bonus point schedu...",150.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052,5ff1e1bb0a720f052300056b,2021-01-03 15:24:43.000,2021-01-03 15:24:43.000,2021-01-03 15:24:43,2021-01-03 15:24:48.000,2021-01-03 15:24:43,2021-01-02 15:24:43
2,5.0,All-receipts receipt bonus,5,1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b,5ff1e1f10a720f052300057a,2021-01-03 15:25:37.000,2021-01-03 15:25:37.000,NaT,2021-01-03 15:25:42.000,NaT,2021-01-03 00:00:00
3,5.0,All-receipts receipt bonus,5.0,4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6,5ff1e1ee0a7214ada100056f,2021-01-03 15:25:34.000,2021-01-03 15:25:34.000,2021-01-03 15:25:34,2021-01-03 15:25:39.000,2021-01-03 15:25:34,2021-01-03 00:00:00
4,5.0,All-receipts receipt bonus,5.0,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052,5ff1e1d20a7214ada1000561,2021-01-03 15:25:06.000,2021-01-03 15:25:06.000,2021-01-03 15:25:11,2021-01-03 15:25:11.000,2021-01-03 15:25:06,2021-01-02 15:25:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603cc0630a720fde100003e6,2021-03-01 10:22:27.000,2021-03-01 10:22:27.000,NaT,2021-03-01 10:22:28.000,NaT,2020-08-17 00:00:00
1115,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603d0b710a720fde1000042a,2021-03-01 15:42:41.873,2021-03-01 15:42:41.873,NaT,2021-03-01 15:42:41.873,NaT,NaT
1116,NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33,603cf5290a720fde10000413,2021-03-01 14:07:37.664,2021-03-01 14:07:37.664,NaT,2021-03-01 14:07:37.664,NaT,NaT
1117,25.0,COMPLETE_NONPARTNER_RECEIPT,25.0,2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33,603ce7100a7217c72c000405,2021-03-01 13:07:28.000,2021-03-01 13:07:28.000,NaT,2021-03-01 13:07:29.000,NaT,2020-08-17 00:00:00


In [197]:
# attempt to break out rewardsReceiptItemList:
pd.json_normalize(receipts, record_path = 'rewardsReceiptItemList', errors='ignore')

# this throws a KeyError, investigating. Error seems to be caused by the fact that the package is expecting just a nested object, rather than a nested list of objects

KeyError: 'rewardsReceiptItemList'

In [200]:
# Alternate package to hopefully handle nested lists better than json_normalize:
# pip install flatten_json
from flatten_json import flatten
receipts_flattened = [flatten(d) for d in receipts]
# receipts_flattened

# Still doesn't handle nested list well... output created distinct columns for each object in the list.

In [69]:
# single out the nested field and create a dataframe & excel file to investigate/play with:
receiptsItemTable = receipts_table['rewardsReceiptItemList']
receiptsItemTable.to_excel(file_path + 'receiptsItem_table.xlsx', sheet_name = 'data', index = False)
receiptsItemTable

0       [{'barcode': '4011', 'description': 'ITEM NOT ...
1       [{'barcode': '4011', 'description': 'ITEM NOT ...
2       [{'needsFetchReview': False, 'partnerItemId': ...
3       [{'barcode': '4011', 'description': 'ITEM NOT ...
4       [{'barcode': '4011', 'description': 'ITEM NOT ...
                              ...                        
1114    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1115                                                  NaN
1116                                                  NaN
1117    [{'barcode': 'B076FJ92M4', 'description': 'mue...
1118                                                  NaN
Name: rewardsReceiptItemList, Length: 1119, dtype: object

In [220]:
# Feel like I've exhausted my knowledge/options to fully blow this table out. Tried some methods in Excel as well as web-powered converter sites.  
# In terms of getting a (mostly) complete field list for purposes of building a data model, cheaty method below:
# Fields are inconsistent across list items, so it's hard to get a complete picture right now. Picking random objects in the list until I find a decent placeholder:
# Overall, I would recommend that this data be stored differently in the first place to avoid this
pd.json_normalize(receipts[5]['rewardsReceiptItemList'], errors='ignore')

,barcode,description,finalPrice,itemPrice,needsFetchReview,originalMetaBriteBarcode,originalMetaBriteDescription,partnerItemId,pointsNotAwardedReason,pointsPayerId,preventTargetGapPoints,quantityPurchased,rewardsGroup,rewardsProductPartnerId,userFlaggedBarcode
0,4011,ITEM NOT FOUND,3.25,3.25,False,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,1,Action not allowed for user and CPG,5332f5fbe4b03c9a25efd0ba,True,1,DORITOS SPICY SWEET CHILI SINGLE SERVE,5332f5fbe4b03c9a25efd0ba,4011
